# 使用 Milvus 和 DeepSeek 构建 RAG（基于民法典的知识库)

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

## 准备工作

### 依赖与环境

In [4]:
!pip install pandas

In [5]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

---

In [6]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
print(api_key)

sk-9edd1c2c3ba24f6f87fc0c78b4a7e385


### 准备数据

我们从 `mfd.md` 加载 markdown 文件。对于文档，我们简单地使用 "# " 来分割文件中的内容，这样可以大致分离出 markdown 文件中每个主要部分的内容。

In [7]:
from glob import glob

text_lines = []

for file_path in glob("mfd.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [8]:
len(text_lines)

30

In [9]:
print(api_key)

sk-9edd1c2c3ba24f6f87fc0c78b4a7e385


### 准备 LLM 和 Embedding 模型

DeepSeek 支持 OpenAI 风格的 API，您可以使用相同的 API 进行微小调整来调用 LLM。

In [10]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

定义一个 embedding 模型，使用 `milvus_model` 来生成文本嵌入。我们以 `DefaultEmbeddingFunction` 模型为例，这是一个预训练的轻量级嵌入模型。

In [11]:
# from pymilvus import model as milvus_model

# embedding_model = milvus_model.DefaultEmbeddingFunction()

from pymilvus import model as milvus_model

# OpenAI国内代理 https://api.apiyi.com/token 
embedding_model = milvus_model.dense.OpenAIEmbeddingFunction(
    model_name='text-embedding-3-large', # Specify the model name
    api_key='sk-GWtxYBmrvJW0JiHtDf745f1582Db489a82BbEcD59e279538', # Provide your OpenAI API key
    base_url='https://api.apiyi.com/v1',
    dimensions=512
)

/Users/wjx/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


生成一个测试嵌入并打印其维度和前几个元素。

In [12]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

512
[-0.02814773  0.00430979 -0.01858036  0.08197533 -0.0316349  -0.05261157
 -0.04882039  0.1247513  -0.02083361  0.0397001 ]


In [13]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])

[-0.00576215  0.02239426 -0.01892634  0.12805134 -0.01240763 -0.07328437
 -0.0028504   0.08630542 -0.04374161  0.03085141]


## 将数据加载到 Milvus

### 创建 Collection

In [15]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "mfd_rag_collection"

关于 `MilvusClient` 的参数：

*   将 `uri` 设置为本地文件，例如 `./milvus.db`，是最方便的方法，因为它会自动利用 Milvus Lite 将所有数据存储在此文件中。
*   如果您有大规模数据，可以在 Docker 或 Kubernetes 上设置性能更高的 Milvus 服务器。在此设置中，请使用服务器 URI，例如 `http://localhost:19530`，作为您的 `uri`。
*   如果您想使用 Zilliz Cloud（Milvus 的完全托管云服务），请调整 `uri` 和 `token`，它们对应 Zilliz Cloud 中的 Public Endpoint 和 Api key。

检查 collection 是否已存在，如果存在则删除它。

In [16]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

创建一个具有指定参数的新 collection。

如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 `id` 字段作为主键，以及一个 `vector` 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

`metric_type` (距离度量类型):
     作用：定义如何计算向量之间的相似程度。
     例如：`IP` (内积) - 值越大通常越相似；`L2` (欧氏距离) - 值越小越相似；`COSINE` (余弦相似度) - 通常转换为距离，值越小越相似。
     选择依据：根据你的嵌入模型的特性和期望的相似性定义来选择。

 `consistency_level` (一致性级别):
     作用：定义数据写入后，读取操作能多快看到这些新数据。
     例如：
         `Strong` (强一致性): 总是读到最新数据，可能稍慢。
         `Bounded` (有界过期): 可能读到几秒内旧数据，性能较好 (默认)。
         `Session` (会话一致性): 自己写入的自己能立刻读到。
         `Eventually` (最终一致性): 最终会读到新数据，但没时间保证，性能最好。
     选择依据：在数据实时性要求和系统性能之间做权衡。

简单来说：
 `metric_type`：怎么算相似。
 `consistency_level`：新数据多久能被读到。

In [17]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

### 插入数据

遍历文本行，创建嵌入，然后将数据插入 Milvus。

这里有一个新字段 `text`，它是在 collection schema 中未定义的字段。它将自动添加到保留的 JSON 动态字段中，该字段在高级别上可以被视为普通字段。

In [18]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|████████████| 30/30 [00:00<00:00, 161112.83it/s]


{'insert_count': 30, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'cost': 0}

## 构建 RAG

### 检索查询数据

我们指定一个关于 Milvus 的常见问题。

In [28]:
question = "不动产登记在哪里办理?"
question1 = "耕地的承包期是多少年?"

在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。

In [31]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)
search_res1 = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question1]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

让我们看一下查询的搜索结果

In [32]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
retrieved_lines_with_distances1 = [
    (res["entity"]["text"], res["distance"]) for res in search_res1[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))
print(json.dumps(retrieved_lines_with_distances1, indent=4))

[
    [
        "\u7b2c\u4e00\u7ae0 \u4e00\u822c\u89c4\u5b9a\n\n**\u7b2c\u4e8c\u767e\u96f6\u56db\u6761** \u4e3a\u4e86\u660e\u786e\u7269\u7684\u5f52\u5c5e\uff0c\u5145\u5206\u53d1\u6325\u7269\u7684\u6548\u7528\uff0c\u4fdd\u62a4\u6743\u5229\u4eba\u7684\u5408\u6cd5\u6743\u76ca\uff0c\u7ef4\u62a4\u793e\u4f1a\u7ecf\u6d4e\u79e9\u5e8f\uff0c\u5236\u5b9a\u672c\u7f16\u3002\n\n**\u7b2c\u4e8c\u767e\u96f6\u4e94\u6761** \u672c\u7f16\u8c03\u6574\u56e0\u7269\u7684\u5f52\u5c5e\u548c\u5229\u7528\u4ea7\u751f\u7684\u6c11\u4e8b\u5173\u7cfb\u3002\n\n**\u7b2c\u4e8c\u767e\u96f6\u516d\u6761** \u56fd\u5bb6\u575a\u6301\u548c\u5b8c\u5584\u793e\u4f1a\u4e3b\u4e49\u516c\u6709\u5236\u4e3a\u4e3b\u4f53\u3001\u591a\u79cd\u6240\u6709\u5236\u7ecf\u6d4e\u5171\u540c\u53d1\u5c55\u7684\u57fa\u672c\u7ecf\u6d4e\u5236\u5ea6\u3002\n\u56fd\u5bb6\u5de9\u56fa\u548c\u53d1\u5c55\u516c\u6709\u5236\u7ecf\u6d4e\uff0c\u9f13\u52b1\u3001\u652f\u6301\u548c\u5f15\u5bfc\u975e\u516c\u6709\u5236\u7ecf\u6d4e\u7684\u53d1\u5c55\u3002\n\u56fd\u5bb6\u5

### 使用 LLM 获取 RAG 响应

将检索到的文档转换为字符串格式。

In [33]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)
context1 = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances1]
)

In [37]:
context

'第一章 一般规定\n\n**第二百零四条** 为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编。\n\n**第二百零五条** 本编调整因物的归属和利用产生的民事关系。\n\n**第二百零六条** 国家坚持和完善社会主义公有制为主体、多种所有制经济共同发展的基本经济制度。\n国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。\n国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。\n\n**第二百零七条** 国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。\n\n**第二百零八条** 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。\n\n**第二百零九条** 不动产物权的设立、变更、转让和消灭，经依法登记，发生效力；未经登记，不发生效力，但是法律另有规定的除外。\n依法属于国家所有的自然资源，所有权可以不登记。\n\n**第二百一十条** 不动产登记，由不动产所在地的登记机构办理。\n国家对不动产实行统一登记制度。统一登记的范围、登记机构和登记办法，由法律、行政法规规定。\n\n**第二百一十一条** 当事人申请登记，应当根据不同登记事项提供材料。\n申请登记材料以及登记事项相关信息，可以公开查询。\n\n**第二百一十二条** 登记机构应当履行下列职责：\n（一）审查申请人提供的材料；\n（二）询问申请人；\n（三）如实、及时登记；\n（四）法律、行政法规规定的其他职责。\n申请登记的不动产存在尚未解决的权属争议的，登记机构应当不予登记，并书面告知申请人。\n\n**第二百一十三条** 登记机构不得有下列行为：\n（一）要求对不动产进行评估；\n（二）以不动产登记为条件收取其他费用；\n（三）超出登记职责范围的其他行为。\n\n**第二百一十四条** 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。\n\n**第二百一十五条** 不动产登记簿由登记机构管理。\n不动产登记簿应当采用纸质形式或者电子形式。\n不动产登记簿采用电子形式的，应当备份。\n\n**第二百一十六条** 不动产登记簿是物权归属和内容的根据。\n不动产登记簿记载的事项与不动产权属证书记载的事项不一致的

In [34]:
context1

'第二节 土地承包经营权\n\n**第三百五十八条** 土地承包经营权人依照法律规定，对集体所有的土地和国家所有由集体经济组织承包的土地，享有占有、使用和收益的权利，有权从事种植业、林业、畜牧业等农业生产。\n\n**第三百五十九条** 土地承包经营权人可以按照法律规定，将土地承包经营权互换、转让。\n当事人通过互换、转让方式取得土地承包经营权的，未经登记，不得对抗善意第三人。\n\n**第三百六十条** 土地承包经营权人依法对承包地享有占有、使用、收益的权利。\n土地承包经营权人可以采取出租、入股或者其他方式流转土地经营权。\n\n**第三百六十一条** 土地承包经营权人可以根据法律规定，将土地承包经营权互换、转让。\n互换、转让后，土地承包经营权人依法享有土地承包经营权。\n\n**第三百六十二条** 土地经营权人有权在合同约定的期限内占有农村土地，开展农业生产经营并取得收益。\n\n**第三百六十三条** 土地经营权人可以依照法律规定，将土地经营权转让、出租、入股或者以其他方式流转。\n未经登记，不得对抗善意第三人。\n\n**第三百六十四条** 耕地的承包期为三十年。草地的承包期为三十年至五十年。林地的承包期为三十年至七十年。\n\n**第三百六十五条** 土地承包经营权人依法享有承包地上的生产经营自主权。\n土地承包经营权人可以采取出租、入股或者其他方式流转土地经营权。\n\n**第三百六十六条** 承包期届满，由土地承包经营权人按照国家有关规定继续承包。\n法律另有规定的，依照其规定。\n\n####\n第六节 地役权\n\n**第三百八十八条** 地役权人有权依照合同约定，利用他人的不动产，以提高自己的不动产的效益。\n前款所称他人的不动产为供役地，自己的不动产为需役地。\n\n**第三百八十九条** 地役权合同一般包括下列条款：\n（一）当事人的姓名或者名称和住所；\n（二）供役地和需役地的位置；\n（三）地役权的目的和期限；\n（四）利用供役地的方式；\n（五）报酬及其支付方式；\n（六）争议解决方式。\n\n**第三百九十条** 设立地役权，当事人应当依照法律规定办理登记。\n未经登记，不得对抗善意第三人。\n\n**第三百九十一条** 地役权人有权依照合同约定，利用供役地，以提高需役地的效益。\n地役权人行使权利，不得损害供役地权利人的合法权益

In [38]:
question

'不动产登记在哪里办理?'

In [39]:
question1

'耕地的承包期是多少年?'

为语言模型定义系统和用户提示。此提示是使用从 Milvus 检索到的文档组装而成的。

In [41]:
SYSTEM_PROMPT = """
Human: 你是一个律师。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。。
<context>
{context}
</context>
<question>
{question}
</question>
"""
USER_PROMPT1 = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。。
<context>
{context1}
</context>
<question>
{question1}
</question>
"""

In [40]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。。\n<context>\n第一章 一般规定\n\n**第二百零四条** 为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编。\n\n**第二百零五条** 本编调整因物的归属和利用产生的民事关系。\n\n**第二百零六条** 国家坚持和完善社会主义公有制为主体、多种所有制经济共同发展的基本经济制度。\n国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。\n国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。\n\n**第二百零七条** 国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。\n\n**第二百零八条** 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。\n\n**第二百零九条** 不动产物权的设立、变更、转让和消灭，经依法登记，发生效力；未经登记，不发生效力，但是法律另有规定的除外。\n依法属于国家所有的自然资源，所有权可以不登记。\n\n**第二百一十条** 不动产登记，由不动产所在地的登记机构办理。\n国家对不动产实行统一登记制度。统一登记的范围、登记机构和登记办法，由法律、行政法规规定。\n\n**第二百一十一条** 当事人申请登记，应当根据不同登记事项提供材料。\n申请登记材料以及登记事项相关信息，可以公开查询。\n\n**第二百一十二条** 登记机构应当履行下列职责：\n（一）审查申请人提供的材料；\n（二）询问申请人；\n（三）如实、及时登记；\n（四）法律、行政法规规定的其他职责。\n申请登记的不动产存在尚未解决的权属争议的，登记机构应当不予登记，并书面告知申请人。\n\n**第二百一十三条** 登记机构不得有下列行为：\n（一）要求对不动产进行评估；\n（二）以不动产登记为条件收取其他费用；\n（三）超出登记职责范围的其他行为。\n\n**第二百一十四条** 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。\n\n**第二百一十五条** 不动产登记簿由登记机构管理。\n不动产登记簿应当采用纸质形式或者电子形式。\n不动产登记簿采用电子形式的，应

In [42]:
USER_PROMPT1

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。。\n<context>\n第二节 土地承包经营权\n\n**第三百五十八条** 土地承包经营权人依照法律规定，对集体所有的土地和国家所有由集体经济组织承包的土地，享有占有、使用和收益的权利，有权从事种植业、林业、畜牧业等农业生产。\n\n**第三百五十九条** 土地承包经营权人可以按照法律规定，将土地承包经营权互换、转让。\n当事人通过互换、转让方式取得土地承包经营权的，未经登记，不得对抗善意第三人。\n\n**第三百六十条** 土地承包经营权人依法对承包地享有占有、使用、收益的权利。\n土地承包经营权人可以采取出租、入股或者其他方式流转土地经营权。\n\n**第三百六十一条** 土地承包经营权人可以根据法律规定，将土地承包经营权互换、转让。\n互换、转让后，土地承包经营权人依法享有土地承包经营权。\n\n**第三百六十二条** 土地经营权人有权在合同约定的期限内占有农村土地，开展农业生产经营并取得收益。\n\n**第三百六十三条** 土地经营权人可以依照法律规定，将土地经营权转让、出租、入股或者以其他方式流转。\n未经登记，不得对抗善意第三人。\n\n**第三百六十四条** 耕地的承包期为三十年。草地的承包期为三十年至五十年。林地的承包期为三十年至七十年。\n\n**第三百六十五条** 土地承包经营权人依法享有承包地上的生产经营自主权。\n土地承包经营权人可以采取出租、入股或者其他方式流转土地经营权。\n\n**第三百六十六条** 承包期届满，由土地承包经营权人按照国家有关规定继续承包。\n法律另有规定的，依照其规定。\n\n####\n第六节 地役权\n\n**第三百八十八条** 地役权人有权依照合同约定，利用他人的不动产，以提高自己的不动产的效益。\n前款所称他人的不动产为供役地，自己的不动产为需役地。\n\n**第三百八十九条** 地役权合同一般包括下列条款：\n（一）当事人的姓名或者名称和住所；\n（二）供役地和需役地的位置；\n（三）地役权的目的和期限；\n（四）利用供役地的方式；\n（五）报酬及其支付方式；\n（六）争议解决方式。\n\n**第三百九十条** 设立地役权，当事人应当依照法律规定办理登记。\n未经登记，不得对抗善意第三人。\n\

使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。

In [43]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
response1 = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT1},
    ],
)
print(response.choices[0].message.content)
print(response1.choices[0].message.content)

根据《中华人民共和国民法典》第二百一十条的规定，不动产登记由不动产所在地的登记机构办理。
根据《中华人民共和国民法典》第三百六十四条的规定，耕地的承包期为三十年。
